In [1]:
from llama_index.core import SimpleDirectoryReader,Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.llms.groq import Groq
from llama_index.core.node_parser import TokenTextSplitter

In [2]:
from importlib.metadata import distributions  
import os, time

for dist in distributions():
    print("%s %s: %s" % (dist.metadata["Name"], dist.version, time.ctime(os.path.getctime(dist._path))))

aiohappyeyeballs 2.4.0: Mon Sep  2 00:19:14 2024
aiohttp 3.10.5: Mon Sep  2 00:20:06 2024
aiosignal 1.3.1: Mon Sep  2 00:19:59 2024
annotated-types 0.7.0: Mon Sep  2 00:19:14 2024
anyio 3.7.0: Wed Jun  7 22:09:57 2023
argon2-cffi 21.3.0: Wed Jun  7 22:10:00 2023
argon2-cffi-bindings 21.2.0: Wed Jun  7 22:09:58 2023
arrow 1.2.3: Wed Jun  7 22:09:58 2023
asgiref 3.8.1: Mon Sep  2 00:19:59 2024
asttokens 2.2.1: Wed Jun  7 22:09:57 2023
async-timeout 4.0.3: Mon Sep  2 00:19:14 2024
attrs 23.1.0: Wed Jun  7 22:09:53 2023
backcall 0.2.0: Wed Jun  7 22:09:42 2023
backoff 2.2.1: Mon Sep  2 00:19:14 2024
bcrypt 4.2.0: Mon Sep  2 00:19:14 2024
beautifulsoup4 4.12.3: Mon Sep  2 00:19:14 2024
bleach 6.0.0: Wed Jun  7 22:09:56 2023
boto3 1.35.10: Mon Sep  2 00:20:23 2024
botocore 1.35.10: Mon Sep  2 00:19:58 2024
build 1.2.1: Mon Sep  2 00:19:57 2024
cachetools 5.5.0: Mon Sep  2 00:19:13 2024
certifi 2024.8.30: Mon Sep  2 00:19:13 2024
cffi 1.15.1: Wed Jun  7 22:09:56 2023
charset-normalizer 3.3.2:

In [1]:
from importlib.metadata import distributions  
import os, time

for dist in distributions():
    print("%s %s: %s" % (dist.metadata["Name"], dist.version, time.ctime(os.path.getctime(dist._path))))

aiohappyeyeballs 2.4.0: Fri Sep 13 23:33:45 2024
aiohttp 3.10.5: Fri Sep 13 23:35:13 2024
aiosignal 1.3.1: Fri Sep 13 23:34:05 2024
annotated-types 0.7.0: Fri Sep 13 23:33:45 2024
anyio 4.4.0: Fri Sep 13 23:34:05 2024
asgiref 3.8.1: Fri Sep 13 23:34:05 2024
asttokens 2.4.1: Fri Sep 13 23:37:44 2024
async-timeout 4.0.3: Fri Sep 13 23:33:45 2024
attrs 24.2.0: Fri Sep 13 23:33:45 2024
backoff 2.2.1: Fri Sep 13 23:33:45 2024
bcrypt 4.2.0: Fri Sep 13 23:33:45 2024
beautifulsoup4 4.12.3: Fri Sep 13 23:34:04 2024
boto3 1.35.18: Fri Sep 13 23:35:45 2024
botocore 1.35.18: Fri Sep 13 23:34:57 2024
build 1.2.2: Fri Sep 13 23:34:04 2024
cachetools 5.5.0: Fri Sep 13 23:33:45 2024
certifi 2024.8.30: Fri Sep 13 23:33:45 2024
charset-normalizer 3.3.2: Fri Sep 13 23:33:45 2024
chromadb 0.5.5: Fri Sep 13 23:35:55 2024
chroma-hnswlib 0.7.6: Fri Sep 13 23:34:04 2024
click 8.1.7: Fri Sep 13 23:34:04 2024
colorama 0.4.6: Fri Sep 13 23:33:45 2024
coloredlogs 15.0.1: Fri Sep 13 23:34:04 2024
comm 0.2.2: Fri S

In [ ]:

for dist in distributions():
    print("%s %s: %s" % (dist.metadata["Name"], dist.version, time.ctime(os.path.getctime(dist._path))))

In [2]:
CHROMA_PATH = "chroma_llama1"
DATA_PATH = "data"

chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
chroma_collection = chroma_client.create_collection("document_collection")

In [6]:
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-8b-8192")
Settings.llm = llm

c:\Users\miriy\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
documents = SimpleDirectoryReader(DATA_PATH).load_data()
db = chromadb.PersistentClient(path=CHROMA_PATH)
chroma_collection = db.get_or_create_collection("document_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [8]:
len(documents)

451

In [9]:
Settings.chunk_size = 1024
Settings.chunk_overlap = 20

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [10]:
retriever = index.as_retriever()

relevant_docs = retriever.retrieve("WHAT are the various steps in NLP?")

relevant_docs

[NodeWithScore(node=TextNode(id_='077b0954-afb0-4465-a170-e9b5342c6a7a', embedding=None, metadata={'page_label': '31', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_type': 'application/pdf', 'file_size': 59957721, 'creation_date': '2024-09-02', 'last_modified_date': '2024-08-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ab6d7e09-8359-41c6-9ff4-bb6cf03727c0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '31', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',

In [6]:
relevant_docs

[NodeWithScore(node=TextNode(id_='bdb17fec-1ba1-4e54-83a8-5a28767e9902', embedding=None, metadata={'page_label': '37', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_type': 'application/pdf', 'file_size': 59957721, 'creation_date': '2024-09-02', 'last_modified_date': '2024-08-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0a5b464f-e9b3-4a94-9c2d-a2b5e35ab6f8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '37', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',

In [11]:
query_engine = index.as_query_engine(similarity_top_k = 10) # similarity_top_k, streaming=True

query_engine

In [12]:
response = query_engine.query("WHAT are the various steps in NLP?")

print(response)

The various steps in NLP include tokenization, lemmatization, stemming, stop word removal, named entity recognition (NER), parts-of-speech (PoS) tagging, and chunking.


In [13]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='077b0954-afb0-4465-a170-e9b5342c6a7a', embedding=None, metadata={'page_label': '31', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_type': 'application/pdf', 'file_size': 59957721, 'creation_date': '2024-09-02', 'last_modified_date': '2024-08-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ab6d7e09-8359-41c6-9ff4-bb6cf03727c0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '31', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',

In [31]:
response = query_engine.query("WHAT IS history of NLP?")

print(response)

The development of NLP as a field of computer science is a relatively recent phenomenon. It has its roots in the 1950s, when the first attempts were made to develop machines that could understand and generate human language.


In [11]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='dd284adc-119a-41bd-9cd9-a2977b5983f1', embedding=None, metadata={'page_label': '2', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_type': 'application/pdf', 'file_size': 59957721, 'creation_date': '2024-09-02', 'last_modified_date': '2024-08-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9b1b5cbc-d982-4c60-9e30-23c0550dde1a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '2', 'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', 'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf', '

In [20]:
response.metadata

{'054d1ad3-b786-4d0d-8d3f-d2a31769bb5d': {'page_label': '37',
  'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_type': 'application/pdf',
  'file_size': 59957721,
  'creation_date': '2024-09-02',
  'last_modified_date': '2024-08-22'},
 '47043163-5ba8-4d0b-92d6-c1c725a4b57e': {'page_label': '141',
  'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_type': 'application/pdf',
  'file_size': 59957721,
  'creation_date': '2024-09-02',
  'last_modified_date': '2024-08-22'},
 '54d3b01c-bc8a-432e-9674-344eb098d1e2': {'page_label': '7',
  'file_name': 'THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_path': 'd:\\Python\\university_chatbot\\data\\THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf',
  'file_type': 'application/pdf',
  'file_size': 59957721,
 